# T-SNE & UMAP
T-SNE (https://lvdmaaten.github.io/tsne/) and UMAP (https://github.com/lmcinnes/umap) are two technicalities, often used by data scientists, that allow to project multivariate data into lower dimensions. They are often used to find clusters in data. I used the fast t-SNE and UMAP implementations offered by Rapids (they require GPU access). 

As stated by the article "How to t-SNE Effectively" (https://distill.pub/2016/misread-tsne/), it is easy to see clusters where there are not, and in our data there are no clear macro clusters, probably. What can be inferedded by t-SNE and UMAP projects, though, is that there are many local clusters, just like in mixture of (this is what can be read from t-SNE). Also there are probably a few outliers, given the noise injected into the artificial data (and this can be read from UMAP, where a few central clusters are surronded by very small scattered ones).

I wonder if, given such evidence, gaussian mixtures may prove a good feature engineering approach.

In [ ]:
%%time
import sys
!cp -f ../input/rapids/rapids.21.06 /opt/conda/envs/rapids.tar.gz
!cd -f /opt/conda/envs/ && tar -xzvf rapids.tar.gz
sys.path = ["/opt/conda/envs/rapids/lib"] + ["/opt/conda/envs/rapids/lib/python3.7"] + ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
!cp -f /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import os
import pprint
import joblib
from functools import partial

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")

# Regressors
import lightgbm as lgb

# Model selection
from sklearn.model_selection import KFold, StratifiedKFold

# Metrics
from sklearn.metrics import mean_squared_error

# Data processing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 
import cudf, cuml
import cupy as cp
from cuml.manifold import TSNE, UMAP
import matplotlib.pyplot as plt
from matplotlib.pyplot import ylim, xlim
%matplotlib inline

In [ ]:
# Loading data 
X = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
X_test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
# Preparing data as a tabular matrix
y = X.claim
X = X.set_index('id').drop('claim', axis='columns')
X_test = X_test.set_index('id')

In [ ]:
# Dealing with missing values
means = X.mean()
X = X.fillna(means)
X_test = X_test.fillna(means)

In [ ]:
tsne = TSNE(n_components=2, perplexity=10, n_neighbors=100)
projection_2D = tsne.fit_transform(X)

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(projection_2D[:,0], projection_2D[:,1],
            c=y.values, 
            edgecolor='none', 
            alpha=0.80, 
            s=10)
plt.axis('off')
plt.show();

In [ ]:
umap = UMAP(n_components=2, n_neighbors=100)
projection_2D = umap.fit_transform(X)

In [ ]:
valid_0 = (projection_2D[:,0] < 20) & (projection_2D[:,0] >-20)
valid_1 = (projection_2D[:,1] < 20) & (projection_2D[:,1] >-20)
valid = valid_0 & valid_1

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(projection_2D[valid, 0], projection_2D[valid, 1],
            c=y.values[valid], 
            edgecolor='none', 
            alpha=0.80, 
            s=10)
plt.axis('off')
plt.show();